# Coding Assistant with crewAI

Source:

Examples for crewAI
https://github.com/crewAIInc/crewAI-examples/tree/main

Notebooks:
https://github.com/crewAIInc/crewAI-examples/tree/main/Notebooks

Notebook (modified):
https://github.com/crewAIInc/crewAI-examples/blob/main/Notebooks/CrewAI%20Flows%20%26%20Langgraph/Coding%20Assistant/coding_assistant_eval.ipynb

## Set environment variables

In [1]:
import os
GROQ_API_KEY = open("/Users/mjack6/.secrets/groq_mjack.apikey", "r").read().strip()
OPENAI_API_KEY = open("/Users/mjack6/.secrets/openai_mjack.apikey", "r").read().strip()
os.environ["GROQ_API_KEY"] = GROQ_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [2]:
# Apply a patch to allow nested asyncio loops in Jupyter
import nest_asyncio
nest_asyncio.apply()

## Load the Documents

In [3]:
from bs4 import BeautifulSoup as Soup
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader

# LCEL docs
url = "https://python.langchain.com/docs/how_to/sequence/#related"
loader = RecursiveUrlLoader(
    url=url, max_depth=20, extractor=lambda x: Soup(x, "html.parser").text
)
docs = loader.load()

In [4]:
# Sort the list based on the URLs and get the text
d_sorted = sorted(docs, key=lambda x: x.metadata["source"])
d_reversed = list(reversed(d_sorted))
concatenated_content = "\n\n\n --- \n\n\n".join(
    [doc.page_content for doc in d_reversed]
)

## Creating Crew

In [5]:
# Importing Crew related components
from crewai import Agent, Task, Crew

# Importing CrewAI Tools
from crewai_tools import WebsiteSearchTool

# Importing Pydantic
from pydantic import BaseModel, Field

class CodeSolution(BaseModel):
  prefix: str = Field(description="Description of the problem and approach")
  imports: str = Field(description="Code block import statements")
  code: str = Field(description="Code block not including import statements")

# Create the coding assistant agent
coding_assistant = Agent(
    role='Coding Assistant',
    goal='Provide accurate and executable code solutions using LCEL',
    backstory="""You are a coding assistant with expertise in LCEL, LangChain expression language. \n
    Here is the LCEL documentation:  \n ------- \n  {context} \n ------- \n
    Answer the user  question based on the \n
    above provided documentation. Ensure any code you provide can be executed with all required imports and variables \n
    defined.""",
    verbose=False,
    llm='gpt-4o'
)

# Create task for code generation
code_generation_task = Task(
    description="""Answer the user question based on the above provided documentation. Ensure any code you provide can be executed
    with all required imports and variables defined. Structure your answer:
    1) a prefix describing the code solution
    2) the imports
    3) the functioning code block

    Your coding task:
    {question}
    """,
    expected_output="Code solution with prefix description, imports, and executable code block",
    agent=coding_assistant,
    output_pydantic=CodeSolution
)

# Create the crew
code_crew = Crew(
    agents=[coding_assistant],
    tasks=[code_generation_task],
    verbose=False
)

/Users/mjack6/GSU_Spring2025/MSA8700/venv_agenticai/lib/python3.11/site-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/Users/mjack6/GSU_Spring2025/MSA8700/venv_agenticai/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
/Users/mjack6/GSU_Spring2025/MSA8700/venv_agenticai/lib/python3.11/site-packages/crewai_tools/tools/scrapegraph_scrape_tool/scrapegraph_scrape_tool.py:34: PydanticDe

In [6]:
# code_crew.train(
#   n_iterations=2,
#   filename="code_crew.pkl",
#   inputs={
#     "question": 'How do I build a RAG chain in LCEL?',
#     "context": str(concatenated_content)
#   }
# )

## Create Flow State

In [7]:
from typing import List

class CodeGenState(BaseModel):
    """
    State for the code generation flow
    """
    error: str = ""
    question: str = ""
    messages: List = []
    generation: str = ""
    iterations: int = 0
    max_iterations: int = 3

## Creating the Code Flow

In [8]:
# Importing CrewAI Flow related components
from crewai.flow.flow import Flow, listen, start, router

class CodeGenFlow(Flow[CodeGenState]):
  def check_code(self):
    print("---CHECKING CODE---")

    code_solution = self.state.generation
    imports = code_solution.imports
    code = code_solution.code

    try:
      exec(imports)
    except Exception as e:
      print("---CODE IMPORT CHECK: FAILED---")
      self.state.error = str(e)
      return "code_failed"

    try:
      exec(imports + "\n" + code)
    except Exception as e:
      print("---CODE BLOCK CHECK: FAILED---")
      self.state.error = str(e)
      return "code_failed"

    print("---NO CODE TEST FAILURES---")
    return "success"

  def fix_code(self):
    if self.state.error != "":
      print("---FIXING CODE---")
      # Create task for fixing code
      code_fix_task = Task(
          description="""You are a coding assistant with expertise in LCEL, LangChain expression language.
          Here is a full set of LCEL documentation:
          -------
          {context}
          -------

          The previous code attempt failed with the following error:
          {error}

          Your coding task:
          {question}

          Previous code attempt:
          {explanation}
          {imports}
          {code}

          Answer with a description of the code solution, followed by the imports, and finally the functioning code block.
          Ensure all imports are correct and the code is executable.""",
          expected_output= "A working code solution to the problem",
          agent=coding_assistant,
          output_pydantic=CodeSolution
      )

      # Create crew for fixing code
      fix_crew = Crew(
          agents=[coding_assistant],
          tasks=[code_fix_task]
      )

      # Execute fix
      result = fix_crew.kickoff(
          inputs={
              "error": self.state.error,
              "question": self.state.question,
              "explanation": self.state.generation.prefix,
              "imports": self.state.generation.imports,
              "code": self.state.generation.code,
              "context": concatenated_content
          }
      )
      self.state.generation = result.pydantic
      self.state.error = ""

  @start()
  def generate_code(self):
    print("---GENERATING CODE SOLUTION---")
    result = code_crew.kickoff(
      inputs={
        "question": self.state.question,
        "context": concatenated_content
      }
    )
    self.state.generation = result.pydantic
    self.state.error = ""

  @router(generate_code)
  def run_check(self):
    result = self.check_code()
    if result != "success":
      return "fix_code"

  @listen('fix_code')
  def run_fix(self):
    self.fix_code()

  @router(run_fix)
  def re_run_check(self):
    result = self.check_code()
    if result != "success":
      return "refix_code"

  @listen('refix_code')
  def re_run_fix(self):
    self.fix_code()

  @listen(re_run_fix)
  def re_re_run_check(self):
    self.check_code()



In [9]:
code_flow = CodeGenFlow()
code_flow.kickoff(inputs={"question": 'How do I build a RAG chain in LCEL?'})
code_flow.state.generation

 
[2025-03-13 15:36:54][🌊 FLOW CREATED: 'CODEGENFLOW']: 2025-03-13 15:36:54.654062
 
[2025-03-13 15:36:54][🤖 FLOW STARTED: 'CODEGENFLOW', 590F0381-7A49-46E0-8193-B34ACA1A30DA]: 2025-03-13 15:36:54.654597
 Flow started with ID: 590f0381-7a49-46e0-8193-b34aca1a30da
 
[2025-03-13 15:36:54][🤖 FLOW METHOD STARTED: 'GENERATE_CODE']: 2025-03-13 15:36:54.654873
---GENERATING CODE SOLUTION---
 
[2025-03-13 15:36:54][🚀 CREW 'CREW' STARTED, 368F5772-413F-4BF5-BD01-847C2CD6098D]: 2025-03-13 15:36:54.655074
 
[2025-03-13 15:36:54][📋 TASK STARTED: ANSWER THE USER QUESTION BASED ON THE ABOVE PROVIDED DOCUMENTATION. ENSURE ANY CODE YOU PROVIDE CAN BE EXECUTED
    WITH ALL REQUIRED IMPORTS AND VARIABLES DEFINED. STRUCTURE YOUR ANSWER:
    1) A PREFIX DESCRIBING THE CODE SOLUTION
    2) THE IMPORTS
    3) THE FUNCTIONING CODE BLOCK

    YOUR CODING TASK:
    HOW DO I BUILD A RAG CHAIN IN LCEL?
    ]: 2025-03-13 15:36:54.660945
 
[2025-03-13 15:36:54][🤖 AGENT 'CODING ASSISTANT' STARTED TASK]: 2025-03-13 

CodeSolution(prefix="To build a Retrieval Augmented Generation (RAG) chain in LCEL, you need to correctly configure and connect each step in your chain. The issue you're experiencing is likely due to a mismatch between expected inputs and the outputs being provided. The RAG chain generally involves a retrieval step followed by some processing and generating step. Hence, it needs appropriate setup for input types and operation flow. Below is a detailed and corrected code example that demonstrates setting up a RAG chain.", imports='from langchain_core.prompts import ChatPromptTemplate\nfrom langchain.chat_models import init_chat_model\nfrom langchain_core.output_parsers import StrOutputParser\nfrom langchain_core.runnables import RunnableParallel', code='prompt = ChatPromptTemplate.from_template("retrieve information about {query}")\nmodel = init_chat_model("llama3-8b-8192", model_provider="groq")\nretriever_chain = prompt | model | StrOutputParser()  # Ensure it matches expected types\n

# Evaluations

We will check for imports, code execution and overall compare with the correct solution.

In [10]:
def check_import(solution) -> dict:
    imports = solution.imports
    try:
        exec(imports)
        return {"key": "import_check", "score": 1}
    except Exception:
        return {"key": "import_check", "score": 0}


def check_execution(solution) -> dict:
    imports = solution.imports
    code = solution.code
    try:
        exec(imports + "\n" + code)
        return {"key": "code_execution_check", "score": 1}
    except Exception:
        return {"key": "code_execution_check", "score": 0}

In [11]:
import pandas as pd

# # Load the evaluation data
# df = pd.read_csv("eval.csv")

data = {'question':
    [
    'How can I use a prompt and model to create a custom RAG example?',  
    'How can I add memory to an arbitrary chain using LangChain?',  
    'How can I make the output of my LCEL chain a string?',  
    'How can I apply a custom function to one of the chains?',  
    'With a RAG chain in LCEL, why are documents required?',    
    'How can I configure the temperature of an LLM?' ,   
    'How can we apply a function call to an LLM in a chain?' ,   
    'How can I run two LCEL chains in parallel and sequentially?' ,   
    'How can I directly pass a string to a runnable chain?',   
    'How can I use a custom function to route between two chains?',   
    'How do I set up a retrieval-augmented generation agent?',   
    'How can I create a LCEL chain that queries a SQL database?',   
    'How do I structure output of an LCEL chain?', 
    ]   
}
df = pd.DataFrame.from_dict(data)

In [12]:
# Store evaluation results
results = []

for _, row in df.iterrows():
    question = row["question"]
    # Run the workflow for each question
    code_flow = CodeGenFlow()
    code_flow.kickoff(inputs={"question": question})

    # Run evaluations
    import_check = check_import(code_flow.state.generation)
    execution_check = check_execution(code_flow.state.generation)

    # Store results
    result = {
        "question": question,
        "import_check": import_check["score"],
        "execution_check": execution_check["score"]
    }
    results.append(result)

# Convert results to dataframe
ca_df = pd.DataFrame(results)
print("\nEvaluation Results:")
print(ca_df)

 
[2025-03-13 15:37:03][🌊 FLOW CREATED: 'CODEGENFLOW']: 2025-03-13 15:37:03.567882
 
[2025-03-13 15:37:03][🤖 FLOW STARTED: 'CODEGENFLOW', 4ED925FE-3884-482C-A71D-BFD077CA3899]: 2025-03-13 15:37:03.568620
 Flow started with ID: 4ed925fe-3884-482c-a71d-bfd077ca3899
 
[2025-03-13 15:37:03][🤖 FLOW METHOD STARTED: 'GENERATE_CODE']: 2025-03-13 15:37:03.568930
---GENERATING CODE SOLUTION---
 
[2025-03-13 15:37:03][🚀 CREW 'CREW' STARTED, 368F5772-413F-4BF5-BD01-847C2CD6098D]: 2025-03-13 15:37:03.569179
 
[2025-03-13 15:37:03][📋 TASK STARTED: ANSWER THE USER QUESTION BASED ON THE ABOVE PROVIDED DOCUMENTATION. ENSURE ANY CODE YOU PROVIDE CAN BE EXECUTED
    WITH ALL REQUIRED IMPORTS AND VARIABLES DEFINED. STRUCTURE YOUR ANSWER:
    1) A PREFIX DESCRIBING THE CODE SOLUTION
    2) THE IMPORTS
    3) THE FUNCTIONING CODE BLOCK

    YOUR CODING TASK:
    HOW CAN I USE A PROMPT AND MODEL TO CREATE A CUSTOM RAG EXAMPLE?
    ]: 2025-03-13 15:37:03.571481
 
[2025-03-13 15:37:03][🤖 AGENT 'CODING ASSISTAN

<string>:6: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/


 
[2025-03-13 15:37:18][✅ LLM CALL COMPLETED]: 2025-03-13 15:37:18.435178
 
[2025-03-13 15:37:18][✅ AGENT 'CODING ASSISTANT' COMPLETED TASK]: 2025-03-13 15:37:18.436596
 
[2025-03-13 15:37:18][✅ TASK COMPLETED: ANSWER THE USER QUESTION BASED ON THE ABOVE PROVIDED DOCUMENTATION. ENSURE ANY CODE YOU PROVIDE CAN BE EXECUTED
    WITH ALL REQUIRED IMPORTS AND VARIABLES DEFINED. STRUCTURE YOUR ANSWER:
    1) A PREFIX DESCRIBING THE CODE SOLUTION
    2) THE IMPORTS
    3) THE FUNCTIONING CODE BLOCK

    YOUR CODING TASK:
    HOW CAN I MAKE THE OUTPUT OF MY LCEL CHAIN A STRING?
    ]: 2025-03-13 15:37:18.436965
 
[2025-03-13 15:37:18][✅ CREW 'CREW' COMPLETED, 368F5772-413F-4BF5-BD01-847C2CD6098D]: 2025-03-13 15:37:18.441982
 
[2025-03-13 15:37:18][👍 FLOW METHOD FINISHED: 'GENERATE_CODE']: 2025-03-13 15:37:18.442236
 
[2025-03-13 15:37:18][🤖 FLOW METHOD STARTED: 'RUN_CHECK']: 2025-03-13 15:37:18.442413
---CHECKING CODE---
Why did the bear go to the doctor?

Because it had a grizzly cough!
---NO

In [13]:
ca_df

,question,import_check,execution_check
0,How can I use a prompt and model to create a c...,1,1
1,How can I add memory to an arbitrary chain usi...,1,0
2,How can I make the output of my LCEL chain a s...,1,1
3,How can I apply a custom function to one of th...,1,1
4,"With a RAG chain in LCEL, why are documents re...",1,1
5,How can I configure the temperature of an LLM?,1,1
6,How can we apply a function call to an LLM in ...,1,1
7,How can I run two LCEL chains in parallel and ...,1,1
8,How can I directly pass a string to a runnable...,1,1
9,How can I use a custom function to route betwe...,1,0


In [14]:
# Evaluation metrics for CrewAI approaches
evaluation_df = pd.DataFrame({
    'Metric': ['Import Check Pass Rate', 'Execution Check Pass Rate'],
    'CrewAI': [
        ca_df['import_check'].mean() * 100,
        ca_df['execution_check'].mean() * 100
    ]
})

# Format percentages to 2 decimal places
evaluation_df['CrewAI'] = evaluation_df['CrewAI'].round(2)

print("\nEvaluation Results (%):")
print(evaluation_df)


Evaluation Results (%):
                      Metric  CrewAI
0     Import Check Pass Rate  100.00
1  Execution Check Pass Rate   69.23
